# Applied Statistical Learning - Prédiction de la durée de mise en chantier en France
Dans le cadre de ce projet, nous avons exploité les données du fichier SITADEL, qui recense de manière exhaustive l'ensemble des permis de construire en France. Pour avoir un échantillon homogène, nous nous sommes concentrés sur la période 2015-2019 (i.e. avant les délais liés au Covid). Nous avons enrichi notre fichier à l'aide de deux sources externes: le fichier complet de l'Insee, agrégeant des données en open data pour l'ensemble des communes, ainsi que la grille de densité des communes de l'Insee. 
Pour faciliter la réplication de nos résultats, nous avons déposé nos données brutes sur une dropbox (1.5Go) : https://www.dropbox.com/scl/fo/b3dw5eht79785mrg7uueq/APQ7m21mQcsZzh7Uy7ASj6k?rlkey=rutboi86sxni30nqg6lzsycqh&st=n60gni1d&dl=0. 

# 1. Mise en place des chemins
Placer l'ensemble des fichiers de la dropbox dans un dossier data, puis adapter le chemin ci-dessous. 

In [ ]:
from pathlib import Path

# CHEMIN A ADAPTER
chemin_data = Path("chemin")   # mettre le chemin local vers le dossier data avec les fichiers de la dropbox

# Suffixes fixes
chemin_autorisation = chemin_data / "Liste-des-autorisations-durbanisme-creant-des-logements.2025-10.csv"
chemin_grilles = chemin_data / "grille_densite_7_niveaux_2019.xlsx"
chemin_dossier = chemin_data / "dossier_complet.csv"

# installer pyarrow pour lire les fichiers parquet
%pip install pyarrow fastparquet
%pip install openpyxl

# 2. Chargement des données
Ce premier chunk permet de sélectionner nos variables d'intérêt, et d'observer notre base avant tout filtre. Le fichier contient initialement 1,8 million de permis de construire, couvrant souvent plusieurs logements. Plusieurs variables d'intérêt sont possibles : la durée avant l'obtention de l'autorisation, la durée entre l'obtention de l'autorisation et la mise en chantier, ou la durée du chantier. Afin d'éviter de nous restreindre à des chantiers terminés, nous étudierons la **durée de mise en chantier**. Après le retrait de durées absentes ou négatives, nous avons 1,7 million de permis. 

In [ ]:
#### CHARGEMENT DES DONNEES ####
# Libraries
import pandas as pd
import numpy as np

# 1. Chargement de Sitadel
# Chemin : défini en amont, vérifier que le chunk a bien été executé. 

# 1.1 Exclusion ex ante des colonnes non pertinentes
all_cols = pd.read_csv(chemin_autorisation, sep=";", skiprows=1, nrows=1).columns.tolist()

vars_mai2022 = [ #On retire les colonnes qui n'existent qu'à partir de mai 2022
    "AN_DEPOT",  # "DPC_PREM", (theoriquement il faudrait la retirer, mais bon)
    "NATURE_PROJET_COMPLETEE",
    "DESTINATION_PRINCIPALE",
    "TYPE_PRINCIP_LOGTS_CREES",
    "TYPE_TRANSFO_PRINCIPAL",
    "TYPE_PRINCIP_LOCAUX_TRANSFORMES",
    "I_PISCINE",
    "I_GARAGE",
    "I_VERANDA",
    "I_ABRI_JARDIN",
    "I_AUTRE_ANNEXE",
    "RES_PERS_AGEES",
    "RES_ETUDIANTS",
    "RES_TOURISME",
    "RES_HOTEL_SOCIALE",
    "RES_SOCIALE",
    "RES_HANDICAPES",
    "RES_AUTRE",
    "NB_LGT_INDIV_PURS",
    "NB_LGT_INDIV_GROUPES",
    "NB_LGT_RES",
    "NB_LGT_COL_HORS_RES",
    "SURF_HEB_TRANSFORMEE",
    "SURF_BUR_TRANSFORMEE",
    "SURF_COM_TRANSFORMEE",
    "SURF_ART_TRANSFORMEE",
    "SURF_IND_TRANSFORMEE",
    "SURF_AGR_TRANSFORMEE",
    "SURF_ENT_TRANSFORMEE",
    "SURF_PUB_TRANSFORMEE",
]
vars_non_pertinentes = [
    "Num_DAU",
    "SIREN_DEM",
    "SIRET_DEM",
    "DENOM_DEM",
    "CODPOST_DEM",
    "LOCALITE_DEM",
    "ADR_NUM_TER",
    "ADR_TYPEVOIE_TER",
    "ADR_LIBVOIE_TER",
    "ADR_LIEUDIT_TER",
    "ADR_LOCALITE_TER",
    "ADR_CODPOST_TER",
    "SEC_CADASTRE1",
    "NUM_CADASTRE1",
    "SEC_CADASTRE2",
    "NUM_CADASTRE2",
    "SEC_CADASTRE3",
    "NUM_CADASTRE3",
]

cols_to_drop = set(vars_mai2022 + vars_non_pertinentes)
use_cols = [c for c in all_cols if c not in cols_to_drop]

# 1.2 Chargement avec les colonnes filtrées
df = pd.read_csv(
    chemin_autorisation,
    sep=";",
    encoding="utf-8",
    skiprows=1,
    usecols=use_cols
)

# 3. Etudions nos dates
date_cols = [
    "DATE_REELLE_AUTORISATION",
    "DATE_REELLE_DOC",
    "DPC_AUT",
    "DATE_REELLE_DAACT",
    "DPC_PREM",
]

for col in date_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Type: {df[col].dtype}")
        print("  Sample values:")
        print(df[col].head(10).tolist())
        print(f"  Null count: {df[col].isna().sum()}")
    else:
        print(f"\n{col}: NOT FOUND in dataframe")


## Puis, nettoyons le dataset
def nettoyer_dataset(df: pd.DataFrame):
    df = df.copy()

    # Conversion des dates en datetime
    # 3 dates en format DD/MM/YYYY
    dmY_cols = ["DATE_REELLE_AUTORISATION", "DATE_REELLE_DOC", "DATE_REELLE_DAACT"]
    for col in dmY_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(
                df[col], errors="coerce", dayfirst=True
            )

    # 2 dates en format YYYY-MM
    for col in ["DPC_AUT", "DPC_PREM"]:
        if col in df.columns:
            # ensure strings and strip whitespace, coerce bad values
            df[col] = pd.to_datetime(
                df[col].astype(str).str.strip(), errors="coerce", format="%Y-%m"
            )

    # On retire les dates fausses
    min_date = pd.Timestamp("1900-01-01") 
    max_date = pd.Timestamp("2025-12-31")
    for col in dmY_cols + ["DPC_AUT", "DPC_PREM"]:
        if col in df.columns:
            mask = (df[col] < min_date) | (df[col] > max_date)
            df.loc[mask, col] = pd.NaT

    # On construit trois variables cibles, même si in fine on n'utilisera que delai_ouverture_chantier
    if "DATE_REELLE_AUTORISATION" in df.columns and "DATE_REELLE_DOC" in df.columns:
        mask = df["DATE_REELLE_AUTORISATION"].notna() & df["DATE_REELLE_DOC"].notna()
        df.loc[mask, "delai_ouverture_chantier"] = (
            df.loc[mask, "DATE_REELLE_DOC"] - df.loc[mask, "DATE_REELLE_AUTORISATION"]
        ).dt.days

    if "DATE_REELLE_DAACT" in df.columns and "DATE_REELLE_DOC" in df.columns:
        mask = df["DATE_REELLE_DAACT"].notna() & df["DATE_REELLE_DOC"].notna()
        df.loc[mask, "duree_travaux"] = (
            df.loc[mask, "DATE_REELLE_DAACT"] - df.loc[mask, "DATE_REELLE_DOC"]
        ).dt.days

    if "DPC_AUT" in df.columns and "DPC_PREM" in df.columns:
        mask = df["DPC_PREM"].notna() & df["DPC_AUT"].notna()
        df.loc[mask, "duree_obtiention_autorisation"] = (
            df.loc[mask, "DPC_AUT"] - df.loc[mask, "DPC_PREM"]
        ).dt.days

    # On traite les variables cibles: conversion en numérique et suppression des valeurs négatives
    duration_cols = [
        "delai_ouverture_chantier",
        "duree_travaux",
        "duree_obtiention_autorisation",
    ]
    for col in duration_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] <= 0, col] = pd.NA

    # Suppression des lignes sans cibles valides (NA, null, zero or negative)
    existing_duration_cols = [c for c in duration_cols if c in df.columns]
    df = df.dropna(subset=existing_duration_cols, how="all")
    
    # On crée mois et année
    df["annee_autorisation"] = df["DATE_REELLE_AUTORISATION"].dt.year
    df["mois_autorisation"] = df["DATE_REELLE_AUTORISATION"].dt.month
    return df


df_clean = nettoyer_dataset(df)

# On règle également les codes
colonnes_codes = [
    "DEP_CODE",
    "COMM",
    "CODGEO",
    "REG_CODE"
]

for col in colonnes_codes:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype("string")


df_clean.to_parquet(
    chemin_data / "autorisations.parquet",
    engine="pyarrow",
    index=False
)

## 4. Comparaison
# Statistics before cleaning
print("Original df shape:", df.shape)
print("Total lines in original df:", len(df))

# Statistics after cleaning
print("\nAfter cleaning:")
print(
    "Lines with non-NA delai_ouverture_chantier:",
    df_clean["delai_ouverture_chantier"].notna().sum(),
)
print("Lines with non-NA duree_travaux:", df_clean["duree_travaux"].notna().sum())
print(
    "Lines with non-NA duree_obtiention_autorisation:",
    df_clean["duree_obtiention_autorisation"].notna().sum(),
)
print(
    "Lines with non-0 duree_obtiention_autorisation:",
    (
        df_clean["duree_obtiention_autorisation"].notna()
        & (df_clean["duree_obtiention_autorisation"] != 0)
    ).sum(),
)
print("\nFinal cleaned df shape:", df_clean.shape)
print("Final cleaned df lines:", len(df_clean))

# 3. Filtre et enrichissement des données
Avec SITADEL, nous avons des données relatives au projet de construction (caractéristiques du demandeur, caractéristiques du projet), mais nous n'avons aucune information sur l'environnement territorial, à l'exception du code commune-département-région. Nous ajoutons donc la grille de densité, nous renseignant sur le type d'occupation du sol (centre urbanisé, rural dense, rural peu dense, ...) et la population communale. Nous ajoutons également des données socio-démographiques du dossier complet : taux de pauvreté communal, part de ménages imposés, nombre de résidences secondaires, etc. 

In [ ]:
# Libraries
import pandas as pd
import openpyxl

# 1. Chargement des autorisations nettoyées
# Chemin a déjà été défini plus haut

df = pd.read_parquet(chemin_data / "autorisations.parquet")

df.info()
print(df.head())

# 2. Chargement de données externes
# 2.1. Grille de densité à 7 niveaux
grille_densite = pd.read_excel(
    chemin_grilles,
    skiprows=4
)
print(grille_densite.head())
grille_densite.info()
print(grille_densite.columns.tolist())

cols_grille = [
    "CODGEO",
    "DENS",# De 1 à 7
    "PMUN17" #Pop° municipale 2017
]
grille_reduite = grille_densite[cols_grille]

# On corrige les formats
df["COMM"] = df["COMM"].astype(str).str.zfill(5)
grille_densite["CODGEO"] = grille_densite["CODGEO"].astype(str).str.zfill(5)
df = df.merge(
    grille_reduite,
    how="left",
    left_on="COMM",
    right_on="CODGEO",
    validate="m:1"
)

missing_rate = df["DENS"].isna().mean()
print(f"Pourcentage sans densité: {missing_rate:.2%}")


# 2.2. Données socio-économiques par communes

all_cols = pd.read_csv(chemin_dossier, sep=";", skiprows=0, nrows=1).columns.tolist()
print(all_cols)

cols_dossier_complet = [
    "CODGEO",
    "TP6021",# taux pauvreté 60% en 2021
    "MED21", #Médiane des revenus fiscaux en 2021"
    "PIMP21", #part de ménages fiscaux imposés
    "PPEN21", #Part des pensions dans le revenu fiscal (proxy pour concentration personnes âgées)
    "DECE1621", #nbr de deces entre 2016 et 2021
    "P16_LOG", #nbr de logements dans la commune en 2022
    "P16_RP", #RP en 2016
    "P16_RSECOCC", #nbr de résidences secondaires et logements occasionnels en 2016
    "P16_LOGVAC", #nbr de logements vacants en 2016 
    "P16_MAISON",
    "P16_APPART",  #Appartements en 2016 
    "P16_NSCOL15P", #Pop 15 ans ou plus non scolarisée en 2016
    "P16_ACTOCC15P", #Actifs occupés 15 ans ou plus en 2016
    "P16_CHOM1564", #Chômeurs 15-64 ans en 2016 (princ);
]

dossier_complet = pd.read_csv(
    chemin_dossier,
    sep=";",
    encoding="utf-8",
    skiprows=0,
    usecols=cols_dossier_complet
)

dossier_complet["CODGEO"] = dossier_complet["CODGEO"].astype(str).str.zfill(5)
df = df.merge(
    dossier_complet,
    how="left",
    left_on="COMM",
    right_on="CODGEO",
    validate="m:1"
)

# 3. Compte des valeurs manquantes
variable_dict = {
    # Identifiers
    "COMM": "Code INSEE de la commune (autorisations)",
    "CODGEO": "Code INSEE de la commune (sources externes)",

    # Grille densité
    "DENS": "Niveau de densité communale (1 = très dense, 7 = très peu dense)",
    "PMUN17": "Population municipale 2017",

    # Socio-éco (INSEE – dossier complet)
    "TP6021": "Taux de pauvreté à 60% du niveau de vie médian (2021)",
    "MED21": "Médiane des revenus fiscaux (€) en 2021",
    "PIMP21": "Part des ménages fiscaux imposés (%)",
    "PPEN21": "Part des pensions dans le revenu fiscal (%)",
    "DECE1621": "Nombre de décès cumulés entre 2016 et 2021",
    "P16_LOG": "Nombre total de logements (2022)",
    "P16_RP": "Nombre de résidences principales (2016)",
    "P16_RSECOCC": "Résidences secondaires et logements occasionnels (2016)",
    "P16_LOGVAC": "Logements vacants (2016)",
    "P16_MAISON": "Maisons individuelles (2016)",
    "P16_APPART": "Appartements (2016)",
    "P16_NSCOL15P": "Population 15+ ans non scolarisée (2016)",
    "P16_ACTOCC15P": "Actifs occupés 15+ ans (2016)",
    "P16_CHOM1564": "Chômeurs 15–64 ans (2016)"
}

vars_added = [v for v in variable_dict.keys() if v in df.columns]

summary_table = (
    pd.DataFrame({
        "variable": vars_added,
        "description": [variable_dict[v] for v in vars_added],
        "share_na": [df[v].isna().mean() for v in vars_added]
    })
    .sort_values("share_na", ascending=False)
    .reset_index(drop=True)
)

print(summary_table)
df_clean.to_parquet(
    chemin_data / "autorisations_enrichies.parquet",
    engine="pyarrow",
    index=False
)

# 4. Prepocessing
Nous pouvons désormais nous intéresser à la plage d'étude (2015-2019), afin de réduire les temps de calcul, et ne pas devoir prendre en compte le bousculement majeur qu'a été le premier confinement.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# A) Préparation des données pour le LASSO

# Variables supplémentaires à retirer du jeu de données final
vars_inutiles_delai_ouverture = [
    "COMM",
    "REG_CODE",
    "REG_LIBELLE",
    "DEP_LIBELLE",
    "NUM_DAU",
    "APE_DEM",
    "CJ_DEM",
    "duree_obtiention_autorisation",
    "DATE_REELLE_AUTORISATION",
    "DATE_REELLE_DAACT",
    "DATE_REELLE_DOC",
    "DPC_AUT",
    "DPC_PREM",
    "DPC_DOC",
    "DPC_DERN", 
    "duree_travaux"]

# Variables à traiter en One-Hot Encoding
vars_categ = [
    "DEP_CODE", 
    "TYPE_DAU", 
    "ETAT_DAU", 
    "CAT_DEM",
    "ZONE_OP",
    "NATURE_PROJET_DECLAREE",
    "UTILISATION",
    "RES_PRINCIP_OU_SECOND",
    "TYP_ANNEXE",
    "RESIDENCE" ]

df = pd.read_parquet(chemin_data / "autorisations_enrichies.parquet")

# Filtrage des régions et des années
regions_outremer = [
    "Guadeloupe", "Martinique", "Guyane",
    "La Réunion", "Mayotte"
]

df["DATE_REELLE_AUTORISATION"] = pd.to_datetime(df["DATE_REELLE_AUTORISATION"], errors="coerce")
df["annee_autorisation"] = df["DATE_REELLE_AUTORISATION"].dt.year
df["mois_autorisation"] = df["DATE_REELLE_AUTORISATION"].dt.month

# Filtrage des lignes sans la variable cible
df_filtre_delai_ouverture = df.dropna(subset=["delai_ouverture_chantier"])
df_filtre_delai_ouverture = df_filtre_delai_ouverture[
    ~df_filtre_delai_ouverture["REG_LIBELLE"].isin(regions_outremer)
]

# On ne conserve que 2015-2019, et on retire les outliers (annulations, et délais >2 ans)
df_filtre_delai_ouverture = df_filtre_delai_ouverture[(df_filtre_delai_ouverture["annee_autorisation"] >= 2015) & (df_filtre_delai_ouverture["annee_autorisation"] <= 2019)]
df_filtre_delai_ouverture = df_filtre_delai_ouverture[df_filtre_delai_ouverture["delai_ouverture_chantier"] <= 600]
df_filtre_delai_ouverture = df_filtre_delai_ouverture[df_filtre_delai_ouverture["ETAT_DAU"] != 4] #on retire les annulations

p95 = df_filtre_delai_ouverture["delai_ouverture_chantier"].quantile(0.95)
p99 = df_filtre_delai_ouverture["delai_ouverture_chantier"].quantile(0.99)

print(f"Seuil 95e percentile : {p95:.1f} jours")
print(f"Seuil 99e percentile : {p99:.1f} jours")

# Nettoyage et conversion des types sur l'échantillon
df_filtre_delai_ouverture = df_filtre_delai_ouverture.drop(columns=vars_inutiles_delai_ouverture, errors="ignore")

# Suppression des lignes sans variable cible
df_model = df_filtre_delai_ouverture.dropna(subset=["delai_ouverture_chantier"])

# On définit X et y
y = df_model["delai_ouverture_chantier"]
X = df_model.drop(columns=["delai_ouverture_chantier"])

# Colonnes numériques
num_cols = X.select_dtypes(
    include=["float", "int", "bool"]
).columns.tolist()

# Colonnes catégorielles
cat_cols = X.select_dtypes(
    include=["string"]
).columns.tolist()

# On gère les NAs des variables explicatives
cols_utiles = num_cols + cat_cols
n_before_na = len(X)
# Suppression des lignes avec NA sur les variables explicatives
X = X.dropna(subset=cols_utiles)
y = y.loc[X.index] 

n_after_na = len(X)
drop_rate = 100 * (1 - n_after_na / n_before_na)

print(f"[INFO] Observations après filtrage NA : {n_after_na:,}")
print(f"[INFO] Taux de suppression des observations : {drop_rate:.2f}%")

# B. Pipeline de pré-traitement et modèle LASSO
preprocess = ColumnTransformer(
    transformers=[
        # Mise à l'échelle des variables numériques (StandardScaler)
        ("num", StandardScaler(), num_cols), 
        # Encodage One-Hot des variables catégorielles (handle_unknown='ignore' pour les valeurs futures non vues)
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ],
    remainder='drop' # On jette ce qui reste
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)


# 5. Régression LASSO


In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1, max_iter=10000)

lasso_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", lasso)
])

lasso_pipeline.fit(X_train, y_train)

y_train_pred = lasso_pipeline.predict(X_train)
y_test_pred = lasso_pipeline.predict(X_test)

test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"RMSE train : {train_rmse:.2f}")
print(f"RMSE test  : {test_rmse:.2f}")
print(f"MAE train  : {train_mae:.2f}")
print(f"MAE test   : {test_mae:.2f}")
print(f"R² train   : {train_r2:.3f}")
print(f"R² test    : {test_r2:.3f}")

print('Classification accuracy on test is: {}'.format(lasso_pipeline.score(X_test, y_test)))


# Nombre de coefficients non nuls
coefs = lasso_pipeline.named_steps["model"].coef_
nb_nonzero = np.sum(coefs != 0)
print("Nombre de coefficients non nuls :", nb_nonzero)

# Liste des coefficients non nuls 
feature_names = list(num_cols) + list(cat_cols)
coefs = lasso_pipeline.named_steps["model"].coef_
nonzero_idx = coefs != 0
selected_features = [(name, coef) for name, coef in zip(feature_names, coefs) if coef != 0]
for name, coef in selected_features:
    print(f"{name}: {coef:.4f}")

## Essayons une version en log 
y_log = np.log1p(y)

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X, y_log,
    test_size=0.2,
    random_state=42
)

lasso_log = Lasso(alpha=0.1, max_iter=10_000)

lasso_log_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", lasso_log)
])
lasso_log_pipeline.fit(X_train2, y_train2)

y_train_log_pred = lasso_log_pipeline.predict(X_train2)
y_test_log_pred = lasso_log_pipeline.predict(X_test2)

train_r2_log = r2_score(y_train2, y_train_log_pred)
test_r2_log = r2_score(y_test2, y_test_log_pred)

print("\n[LASSO – log(durée + 1)]")
print(f"R² train   : {train_r2_log:.3f}")
print(f"R² test    : {test_r2_log:.3f}")

# 6. Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestRegressor(
        max_depth=12, 
        min_samples_split=10,
        min_samples_leaf=4,
        n_estimators=200,
        n_jobs=-1,
        random_state=42
    ))
]) #On peut modifier la profondeur peut être pour améliorer les résultats 

rf_pipeline.fit(X_train, y_train)

y_train_pred_rf = rf_pipeline.predict(X_train)
y_test_pred_rf  = rf_pipeline.predict(X_test)

train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
test_rmse_rf  = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))

train_mae_rf = mean_absolute_error(y_train, y_train_pred_rf)
test_mae_rf  = mean_absolute_error(y_test, y_test_pred_rf)

train_r2_rf = r2_score(y_train, y_train_pred_rf)
test_r2_rf  = r2_score(y_test, y_test_pred_rf)

print(f"RMSE train : {train_rmse_rf:.2f}")
print(f"RMSE test  : {test_rmse_rf:.2f}")
print(f"MAE train  : {train_mae_rf:.2f}")
print(f"MAE test   : {test_mae_rf:.2f}")
print(f"R² train   : {train_r2_rf:.3f}")
print(f"R² test    : {test_r2_rf:.3f}")

print('Classification accuracy on test is: {}'.format(rf_pipeline.score(X_test, y_test)))


# 7. Réseau de neurones

In [ ]:
from sklearn.neural_network import MLPRegressor

mlp_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", MLPRegressor(
        hidden_layer_sizes=(32, 16),   
        activation="relu",
        solver="adam",
        alpha=1e-4,
        learning_rate="adaptive",
        learning_rate_init=0.001,
        batch_size=256,               
        max_iter=300,                  
        early_stopping=True,
        n_iter_no_change=15,
        tol=1e-4,
        random_state=42
    ))
])

mlp_pipeline.fit(X_train, y_train)

# Prédictions
y_train_pred_mlp = mlp_pipeline.predict(X_train)
y_test_pred_mlp  = mlp_pipeline.predict(X_test)

# Métriques
train_rmse_mlp = np.sqrt(mean_squared_error(y_train, y_train_pred_mlp))
test_rmse_mlp  = np.sqrt(mean_squared_error(y_test, y_test_pred_mlp))

train_mae_mlp = mean_absolute_error(y_train, y_train_pred_mlp)
test_mae_mlp  = mean_absolute_error(y_test, y_test_pred_mlp)

train_r2_mlp = r2_score(y_train, y_train_pred_mlp)
test_r2_mlp  = r2_score(y_test, y_test_pred_mlp)

print(f"[MLP] RMSE train : {train_rmse_mlp:.2f}")
print(f"[MLP] RMSE test  : {test_rmse_mlp:.2f}")
print(f"[MLP] MAE train  : {train_mae_mlp:.2f}")
print(f"[MLP] MAE test   : {test_mae_mlp:.2f}")
print(f"[MLP] R² train   : {train_r2_mlp:.3f}")
print(f"[MLP] R² test    : {test_r2_mlp:.3f}")



# 8. SVR

In [ ]:
from sklearn.svm import LinearSVR

svr_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LinearSVR(C=1.0, epsilon=1.0, random_state=0))
])

svr_pipeline.fit(X_train, y_train)

y_train_pred_svr = svr_pipeline.predict(X_train)
y_test_pred_svr  = svr_pipeline.predict(X_test)

train_rmse_svr = np.sqrt(mean_squared_error(y_train, y_train_pred_svr))
test_rmse_svr  = np.sqrt(mean_squared_error(y_test, y_test_pred_svr))

train_mae_svr = mean_absolute_error(y_train, y_train_pred_svr)
test_mae_svr  = mean_absolute_error(y_test, y_test_pred_svr)

train_r2_svr = r2_score(y_train, y_train_pred_svr)
test_r2_svr  = r2_score(y_test, y_test_pred_svr)

print(f"RMSE train : {train_rmse_svr:.2f}")
print(f"RMSE test  : {test_rmse_svr:.2f}")
print(f"MAE train  : {train_mae_svr:.2f}")
print(f"MAE test   : {test_mae_svr:.2f}")
print(f"R² train   : {train_r2_svr:.3f}")
print(f"R² test    : {test_r2_svr:.3f}")

#print('Returned hyperparameter: {}'.format(svr_pipeline.best_params_))
#print('Best classification accuracy in train is: {}'.format(svr_pipeline.best_score_))
print('Classification accuracy on test is: {}'.format(svr_pipeline.score(X_test, y_test)))
